<a href="https://colab.research.google.com/github/samgregson/GHPT-colab-experiments/blob/main/Grasshopper_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup
install openai and setup API key

This notebook works both in a Colab environment and on local machine

Colab:
- API key must be saved in Colab sectrets as OPENAI_API_KEY

Local:
- API key must be defined in the .env file (refer to example.env)

In [2]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
IN_COLAB

False

In [32]:
if IN_COLAB:
    !pip install openai
    !pip install requests

from openai import OpenAI

if IN_COLAB:
    from google.colab import userdata
    OPENAI_API_KEY=userdata.get("OPENAI_API_KEY")
else:
    import os
    from dotenv import load_dotenv
    load_dotenv()
    OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

### load files into Colab

NOTE: you must refresh your Colab files directory to see changes

In [3]:
if IN_COLAB:
    !git clone "https://github.com/samgregson/GHPT-experiments"
    import json
    with open('/content/GHPT-experiments/components.json', 'r') as f:
        components_dict = json.load(f)
    print(components_dict)

# OpenAI

In [71]:
from patch_openai import patch_openai

client = OpenAI(api_key=OPENAI_API_KEY)

if not IN_COLAB:
    client = patch_openai(client)

In [79]:
# create the openai api call function
def call_openai(prompt: str, system_prompt: str = "", model: str = "gpt-3.5-turbo-1106", temperature: float = 0):
    if system_prompt == "":
        messages=[
            {"role": "user", "content": prompt}
        ]
    else:
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ]

    completion = client.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=messages
    )

    return completion

test connection

In [75]:
model: str = "gpt-3.5-turbo-1106"
temperature: float = 0

prompt="say this is a test"
system_prompt=""

completion = call_openai(prompt=prompt, system_prompt=system_prompt, model=model, temperature=temperature)
result = completion.choices[0].message.content
print(result)

This is a test.


# GHPT

## prompt template

In [36]:
system_prompt = """
You are a Grasshopper Expert and are going to help create Grasshopper Definitions.
Keep the answers short and concise.
Make sure you create and connect a component for every non-optional input
If you're not sure about the answer, but think there's additional information that could help you, please ask for that information.

Always use the given format, avoid any devitation.
"""

In [37]:
prompt_template = r"""
// Question : How do I add two numbers inside of Grasshopper?
// Reasoning: To add two numbers we need to Add. There is an Addition component that performs this function. We need to create two numbers the user can edit, we can use the Number Slider for both numbers. And then we can Connections all of the components together
// JSON:
{
	"Advice": "Make sure to set the number sliders to the correct value",
	"Additions": [
		{
			"Name": "Number Slider",
			"Id": 1, // This number must be unique
			"value": "0..25..100" // This is optional
		},
		{
			"Name": "Number Slider",
			"Id": 2,
			"value": "-50..25..100" // This is optional
		},
		{
			"Name": "Addition",
			"Id": 3
		}
	],
	"Connections": [
		{
			"To": { // Addition
				"Id": 3,
				"ParameterName": "A"
			},
			"From": { // Number Slider,
				"Id": 1,
				"ParameterName": "number"
			}
		},
		{
			"To": { // Addition
				"Id": 3,
				"ParameterName": "B"
			},
			"From": { // Number Slider,
				"Id": 2,
				"ParameterName": "number"
			}
		}
	]
}

// Question: How do I create a cup shape in grasshopper
// Reasoning: To create a cup shape in Grasshopper, we"ll first create a circle using a "Circle CNR" component, which will act as the base of the cup. Then, we will use a "Move" component to move the base circle vertically to create the upper rim of the cup. To create the body of the cup, we"ll loft these two circles using a "Loft" component.
// JSON:
{
	"Advice": "Remember, to properly define the cup\"s dimensions using the number slider",
	"Additions": [
		{
			"Name": "Circle CNR",
			"Id": 1
		},
		{
			"Name": "Move",
			"Id": 2
		},
		{
			"Name": "Loft",
			"Id": 3
		}
	],
	"Connections": [
		{
			"To": {
				"Id": 2,
				"ParameterName": "Geometry"
			},
			"From": {
				"Id": 1,
				"ParameterName": "Circle"
			}
		},
		{
			"To": {
				"Id": 3,
				"ParameterName": "Curves"
			},
			"From": {
				"Id": 1,
				"ParameterName": "Circle"
			}
		},
		{
			"To": {
				"Id": 3,
				"ParameterName": "Curves"
			},
			"From": {
				"Id": 2,
				"ParameterName": "Geometry"
			}
		}
	]
}
```

// Question: How do I create a twisty skyscraper?
// Reasoning: We can create the "twist" using the twist component, so lets extrude a rectangle and twist it!
// JSON:
{
	"Advice": "Make sure to use reasonable inputs or the skyscraper will look weird",
	"Additions": [
		{
			"Name": "Rectangle",
			"Id": 1
		},
		{
			"Name": "Number Slider",
			"Id": 2,
			"Value": "0..50..100"
		},
		{
			"Name": "Unit Z",
			"Id": 3
		},
		{
			"Name": "Extrusion",
			"Id": 4
		},
		{
			"Name": "Number Slider",
			"Id": 5,
			"Value": "0..90..360"
		},
		{
			"Name": "Line",
			"Id": 6
		},
		{
			"Name": "Point"
			"Id": 7,
			"value": "{0,0,0}"
		},
		{
			"Name": "Point"
			"Id": 8,
			"value": "{0,0,250}"
		},
		{
			"Name": "Twist",
			"Id": 9
		},
		{
			"Name": "Solid Union",
			"Id": 10
		},
		{
			"Name": "Brep Join",
			"Id": 11
		}
	],
	"Connections": [
		{
			"To": {
				"Id": 4,
				"ParameterName": "Base"
			},
			"From": {
				"Id": 1,
				"ParameterName": "Rectangle"
			}
		},
		{
			"To": {
				"Id": 3,
				"ParameterName": "Factor"
			},
			"From": {
				"Id": 2,
				"ParameterName": "Number"
			}
		},
		{
			"To": {
				"Id": 9,
				"ParameterName": "Angle"
			},
			"From": {
				"Id": 5,
				"ParameterName": "Number"
			}
		},
		{
			"To": {
				"Id": 4,
				"ParameterName": "Direction"
			},
			"From": {
				"Id": 3,
				"ParameterName": "Unit vector"
			}
		},
		{
			"To": {
				"Id": 9,
				"ParameterName": "Geometry"
			},
			"From": {
				"Id": 4,
				"ParameterName": "Extrusion"
			}
		},
		{
			"To": {
				"Id": 6,
				"ParameterName": "Start Point"
			},
			"From": {
				"Id": 7,
				"ParameterName": "Point"
			}
		},
		{
			"To": {
				"Id": 6,
				"ParameterName": "End Point"
			},
			"From": {
				"Id": 8,
				"ParameterName": "Point"
			}
		},
		{
			"To": {
				"Id": 10,
				"ParameterName": "Breps"
			},
			"From": {
				"Id": 9,
				"ParameterName": "Geometry"
			}
		},
		{
			"To": {
				"Id": 11,
				"ParameterName": "Breps"
			},
			"From": {
				"Id": 10,
				"ParameterName": "Result"
			}
		},
		{
      "To": {
          "Id": 9,
          "ParameterName": "Axis"
        },
      "From": {
          "Id": 6,
          "ParameterName": "Line"
      }
		}
	]
}

// Question: {QUESTION}
// Reasoning:
"""

In [38]:
prompt_template = prompt_template.replace("{","{{").replace("}","}}").replace("{{QUESTION}}","{QUESTION}")

In [39]:
prompt_template_error = """The above json returned the error: {ERROR}
re-evaluating the initial task and correcting for the json error the correct json should be

// JSON:
"""

In [76]:
prompt_template.format(QUESTION="create a sphere")

'\n// Question : How do I add two numbers inside of Grasshopper?\n// Reasoning: To add two numbers we need to Add. There is an Addition component that performs this function. We need to create two numbers the user can edit, we can use the Number Slider for both numbers. And then we can Connections all of the components together\n// JSON:\n{\n\t"Advice": "Make sure to set the number sliders to the correct value",\n\t"Additions": [\n\t\t{\n\t\t\t"Name": "Number Slider",\n\t\t\t"Id": 1, // This number must be unique\n\t\t\t"value": "0..25..100" // This is optional\n\t\t},\n\t\t{\n\t\t\t"Name": "Number Slider",\n\t\t\t"Id": 2,\n\t\t\t"value": "-50..25..100" // This is optional\n\t\t},\n\t\t{\n\t\t\t"Name": "Addition",\n\t\t\t"Id": 3\n\t\t}\n\t],\n\t"Connections": [\n\t\t{\n\t\t\t"To": { // Addition\n\t\t\t\t"Id": 3,\n\t\t\t\t"ParameterName": "A"\n\t\t\t},\n\t\t\t"From": { // Number Slider,\n\t\t\t\t"Id": 1,\n\t\t\t\t"ParameterName": "number"\n\t\t\t}\n\t\t},\n\t\t{\n\t\t\t"To": { // Additi

## Baseline Call

In [78]:
# model: str = "gpt-3.5-turbo-1106"
model: str = "gpt-3.5-turbo"
# model = "gpt-4"
temperature: float = 0

prompt=system_prompt + prompt_template.format(QUESTION="sphere")

completion = call_openai(prompt=prompt, system_prompt="", model=model, temperature=temperature)
result = completion.choices[0].message
print(result.content)

To create a sphere in Grasshopper, we can use the "Sphere" component. We can define the center point of the sphere using a "Point" component and the radius of the sphere using a "Number Slider" component. 

// JSON:
{
	"Advice": "Make sure to set the number slider to the desired radius of the sphere",
	"Additions": [
		{
			"Name": "Sphere",
			"Id": 1
		},
		{
			"Name": "Point",
			"Id": 2,
			"value": "{0,0,0}" // This is optional
		},
		{
			"Name": "Number Slider",
			"Id": 3,
			"value": "0..5..10" // This is optional
		}
	],
	"Connections": [
		{
			"To": { // Sphere
				"Id": 1,
				"ParameterName": "Center"
			},
			"From": { // Point
				"Id": 2,
				"ParameterName": "Point"
			}
		},
		{
			"To": { // Sphere
				"Id": 1,
				"ParameterName": "Radius"
			},
			"From": { // Number Slider
				"Id": 3,
				"ParameterName": "Number"
			}
		}
	]
}
